In [1]:
import pandas as pd
import pymongo
from nba_api.stats.static import teams

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.sportsDB

In [23]:
nfl_df = pd.read_csv("matchups_data.csv")
nfl_df = nfl_df.rename(columns={"TEAM NAME": "TEAM_NAME"})
nfl_df.sort_values(by="TEAM_NAME")

,TEAM_NAME,ref,MATCHUP,LOCATION,WL,OPPONENT,ref2
1567,Arizona Cardinals,Arizona Cardinals,Arizona Cardinals vs. Minnesota Vikings,Home,W,Minnesota Vikings,Minnesota Vikings
1645,Arizona Cardinals,Arizona Cardinals,Arizona Cardinals @ New England Patriots,Away,W,New England Patriots,New England Patriots
1646,Arizona Cardinals,Arizona Cardinals,Arizona Cardinals vs. New England Patriots,Home,L,New England Patriots,New England Patriots
2478,Arizona Cardinals,Arizona Cardinals,Arizona Cardinals vs.Tennessee Titans,Home,L,Tennessee Titans,Tennessee Titans
2477,Arizona Cardinals,Arizona Cardinals,Arizona Cardinals @Tennessee Titans,Away,W,Tennessee Titans,Tennessee Titans
1745,Arizona Cardinals,Arizona Cardinals,Arizona Cardinals @ New Orleans Saints,Away,L,New Orleans Saints,New Orleans Saints
1746,Arizona Cardinals,Arizona Cardinals,Arizona Cardinals vs. New Orleans Saints,Home,W,New Orleans Saints,New Orleans Saints
836,Arizona Cardinals,Arizona Cardinals,Arizona Cardinals vs. Detroit Lions,Home,L,Detroit Lions,Detroit Lions
835,Arizona Cardinals,Arizona Cardinals,Arizona Cardinals @ Detroit Lions,Away,W,Detroit Lions,Detroit Lions
746,Arizona Cardinals,Arizona Cardinals,Arizona Cardinals vs. Denver Broncos,Home,W,Denver Broncos,Denver Broncos


In [50]:
grouped_df = nfl_df.groupby(['TEAM_NAME', 'OPPONENT','LOCATION', 'WL'], as_index=False)
winloss = grouped_df.count()
# winloss = winloss.sort_values("TEAM_NAME")
winloss = winloss.sort_values(['TEAM_NAME', "OPPONENT"])
winloss

,TEAM_NAME,OPPONENT,LOCATION,WL,ref,MATCHUP,ref2
0,Arizona Cardinals,Atlanta Falcons,Away,L,3,3,3
1,Arizona Cardinals,Atlanta Falcons,Home,W,3,3,3
2,Arizona Cardinals,Buffalo Bills,Away,L,1,1,1
3,Arizona Cardinals,Buffalo Bills,Home,W,1,1,1
4,Arizona Cardinals,Carolina Panthers,Away,L,2,2,2
5,Arizona Cardinals,Carolina Panthers,Home,W,2,2,2
6,Arizona Cardinals,Chicago Bears,Away,W,1,1,1
7,Arizona Cardinals,Chicago Bears,Home,L,1,1,1
8,Arizona Cardinals,Cleveland Browns,Away,W,1,1,1
9,Arizona Cardinals,Cleveland Browns,Home,L,1,1,1


In [51]:
team_list = nfl_df['TEAM_NAME'].unique().tolist()

In [52]:
teamname = "Arizona Cardinals"
opponent = "Atlanta Falcons"
superdict = {}
regdict = {}
littledict = {}
wins = 0
losses = 0
for index, row in winloss.iterrows():
    
    if row['TEAM_NAME'] == teamname:
        if row['OPPONENT'] == opponent:
            if row['LOCATION'] == 'Away':
                if row['WL'] == "W":
                    wins = row['MATCHUP']
                if row['WL'] == 'L':
                    losses = row['MATCHUP']
        if row['OPPONENT'] != opponent:
            littledict['wins'] = wins
            littledict['losses'] = losses
            regdict[opponent] = littledict
            opponent = row['OPPONENT']
            wins = 0
            losses = 0
            littledict = {}
            if row['LOCATION'] == 'Away':
                if row['WL'] == "W":
                    wins = row['MATCHUP']
                if row['WL'] == 'L':
                    losses = row['MATCHUP']
    elif row['TEAM_NAME'] != teamname:
        littledict['wins'] = wins
        littledict['losses'] = losses
        regdict[opponent] = littledict
        superdict[teamname] = regdict
        regdict = {}
        littledict = {}
        wins = 0
        losses = 0
        teamname = row['TEAM_NAME']
        opponent = row['OPPONENT']
        
        if row['WL'] == "W":
            wins = row['MATCHUP']
        if row['WL'] == 'L':
            losses = row['MATCHUP']
littledict['wins'] = wins
littledict['losses'] = losses
regdict[opponent] = littledict
superdict[teamname] = regdict

In [53]:
superdict['Detroit Lions']

{'Arizona Cardinals': {'wins': 0, 'losses': 3},
 'Carolina Panthers': {'wins': 0, 'losses': 1},
 'Chicago Bears': {'wins': 3, 'losses': 2},
 'Cleveland Browns': {'wins': 1, 'losses': 0},
 'Dallas Cowboys': {'wins': 0, 'losses': 2},
 'Green Bay Packers': {'wins': 1, 'losses': 4},
 'Houston Texans': {'wins': 0, 'losses': 1},
 'Indianapolis Colts': {'wins': 1, 'losses': 0},
 'Jacksonville Jaguars': {'wins': 1, 'losses': 0},
 'Kansas City Chiefs': {'wins': 0, 'losses': 1},
 'Los Angeles Chargers': {'wins': 0, 'losses': 1},
 'Los Angeles Rams': {'wins': 0, 'losses': 1},
 'Minnesota Vikings': {'wins': 2, 'losses': 3},
 'New England Patriots': {'wins': 0, 'losses': 1},
 'New Orleans Saints': {'wins': 2, 'losses': 0},
 'New York Giants': {'wins': 0, 'losses': 1},
 'New York Jets': {'wins': 1, 'losses': 0},
 'Philadelphia Eagles': {'wins': 1, 'losses': 1},
 'Pittsburgh Steelers': {'wins': 0, 'losses': 1},
 'San Francisco 49ers': {'wins': 0, 'losses': 1},
 'Seattle Seahawks': {'wins': 0, 'losses

In [28]:
del superdict['Arizona Cardinals']
superdict

{'Atlanta Falcons': {'Oakland Raiders': {'wins': 1, 'losses': 0},
  'New York Giants': {'wins': 0, 'losses': 1},
  'Minnesota Vikings': {'wins': 0, 'losses': 1},
  'New Orleans Saints': {'wins': 0, 'losses': 0},
  'Philadelphia Eagles': {'wins': 0, 'losses': 0},
  'Washington Redskins': {'wins': 0, 'losses': 0},
  'San Francisco 49ers': {'wins': 0, 'losses': 2},
  'Seattle Seahawks': {'wins': 0, 'losses': 1},
  'Tampa Bay Buccaneers': {'wins': 3, 'losses': 2},
  'Tennessee Titans': {'wins': 1, 'losses': 0},
  'Miami Dolphins': {'wins': 0, 'losses': 1},
  'Detroit Lions': {'wins': 1, 'losses': 0},
  'Los Angeles Rams': {'wins': 1, 'losses': 0},
  'Arizona Cardinals': {'wins': 0, 'losses': 1},
  'Baltimore Ravens': {'wins': 0, 'losses': 1},
  'Buffalo Bills': {'wins': 1, 'losses': 0},
  'Carolina Panthers': {'wins': 2, 'losses': 3},
  'Cincinnati Bengals': {'wins': 0, 'losses': 1},
  'Dallas Cowboys': {'wins': 1, 'losses': 0},
  'Denver Broncos': {'wins': 1, 'losses': 0},
  'Los Angeles 

In [54]:
nfl_coll = db.nflmatchupdict

In [55]:
nfl_coll.insert(superdict)

C:\Users\Will\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  """Entry point for launching an IPython kernel.


ObjectId('5c772a889ec56b2bfc4ea206')

ref  MATCHUP  ref2
TEAM_NAME           OPPONENT             LOCATION WL                    
Arizona Cardinals   Atlanta Falcons      Away     L     3        3     3
                                         Home     W     3        3     3
                    Buffalo Bills        Away     L     1        1     1
                                         Home     W     1        1     1
                    Carolina Panthers    Away     L     2        2     2
                                         Home     W     2        2     2
                    Chicago Bears        Away     W     1        1     1
                                         Home     L     1        1     1
                    Cleveland Browns     Away     W     1        1     1
                                         Home     L     1        1     1
                    Dallas Cowboys       Away     W     1        1     1
                                         Home     L     1        1     1
                    Denver Broncos       Away     L     1        1     1
                                         Home     W     1        1     1
                    Detroit Lions        Away     W     1        1     1
                                         Home     L     1        1     1
                    Green Bay Packers    Away     L     1        1     1
                                         Home     W     1        1     1
                    Jacksonville Jaguars Away     W     1        1     1
                                         Home     L     1        1     1
                    Los Angeles Rams     Away     L     2        2     2
                                                  W     3        3     3
                                         Home     L     3        3     3
                                                  W     2        2     2
                    Miami Dolphins       Away     L     1        1     1
                                         Home     W     1        1     1
                    Minnesota Vikings    Away     L     2        2     2
                                         Home     W     2        2     2
                    New England Patriots Away     W     1        1     1
                                         Home     L     1        1     1
...                                                   ...      ...   ...
Washington Redskins Houston Texans       Away     L     1        1     1
                                         Home     W     1        1     1
                    Indianapolis Colts   Away     L     1        1     1
                                         Home     W     1        1     1
                    Los Angeles Rams     Away     L     1        1     1
                                         Home     W     1        1     1
                    Minnesota Vikings    Away     L     2        2     2
                                         Home     W     2        2     2
                    New England Patriots Away     L     1        1     1
                                         Home     W     1        1     1
                    New Orleans Saints   Away     W     1        1     1
                                         Home     L     1        1     1
                    New York Giants      Away     L     4        4     4
                                                  W     1        1     1
                                         Home     L     1        1     1
                                                  W     4        4     4
                    New York Jets        Away     L     1        1     1
                                         Home     W     1        1     1
                    Oakland Raiders      Away     W     1        1     1
                                         Home     L     1        1     1
                    Philadelphia Eagles  Away     L     2        2     2
                                                  W     3        3     3
                                       

In [49]:

winloss.loc[winloss['TEAM_NAME'] == 'Detroit Lions']

,TEAM_NAME,OPPONENT,LOCATION,WL,ref,MATCHUP,ref2
574,Detroit Lions,Arizona Cardinals,Away,L,3,3,3
575,Detroit Lions,Arizona Cardinals,Home,W,3,3,3
576,Detroit Lions,Carolina Panthers,Away,L,1,1,1
577,Detroit Lions,Carolina Panthers,Home,W,1,1,1
578,Detroit Lions,Chicago Bears,Away,L,2,2,2
579,Detroit Lions,Chicago Bears,Away,W,3,3,3
580,Detroit Lions,Chicago Bears,Home,L,3,3,3
581,Detroit Lions,Chicago Bears,Home,W,2,2,2
582,Detroit Lions,Cleveland Browns,Away,W,1,1,1
583,Detroit Lions,Cleveland Browns,Home,L,1,1,1
